In [63]:
import cv2
import numpy as np


input_path = "Ulcer_2024-08-07-08-25-26_23438.bmp"

image = cv2.imread(input_path)

In [64]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#obszar różny od czarnego
mask = np.where(gray > 10, 255, 0).astype(np.uint8)  
# Znalezienie konturów
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#największy kontur <- ramka obrazka
largest_contour = max(contours, key=cv2.contourArea)
(x, y), radius = cv2.minEnclosingCircle(largest_contour)
center = (int(x), int(y))
radius = int(radius)

circle_mask = np.zeros_like(gray)
cv2.circle(circle_mask, center, radius, 255, thickness=-1)

result = cv2.bitwise_and(image, image, mask=circle_mask)

In [65]:
# Konwersja do skali szarości
gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# Progowanie – znajdowanie czarnych regionów
_, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)

# Inpainting (wypełnianie braków)
output = cv2.inpaint(result, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

# 🔹 Zapisz wynik do pliku
cv2.imwrite("obraz_wypelniony.bmp", output)


True

In [66]:
import os

base, ext = os.path.splitext(input_path)  # Rozdzielenie nazwy i rozszerzenia
output_path = f"blank{base}{ext}" 

# Zapisz wynik
cv2.imwrite(output_path, output)

print(f"Zapisano plik jako: {output_path}")

Zapisano plik jako: blankUlcer_2024-08-07-08-25-26_23438.bmp


In [28]:

# Konwersja do skali szarości
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Wykrywanie okręgu – znajdź największy biały obszar
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
                           param1=50, param2=30, minRadius=100, maxRadius=500)

# Utworzenie maski – czarne tło, biały okrąg
mask = np.zeros_like(gray)

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        cv2.circle(mask, (i[0], i[1]), i[2], (255, 255, 255), thickness=-1)  # Biały wypełniony okrąg

# Zastosowanie maski
result = cv2.bitwise_and(image, image, mask=mask)

# Zapisz wynik
cv2.imwrite("obraz_w_czesciowym_kole.bmp", result)


True